
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>


# Generate Tokens
## Introduction

In the next few notebooks, we will use the Databricks CLI to run code from a notebook, in addition to using the UI. Since we are in a learning environment, we will save a credentials file right here in the workspace. In a production environment, follow your organization's security policies for storing credentials.



## REQUIRED - SELECT CLASSIC COMPUTE
Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default.
Follow these steps to select the classic compute cluster:
1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.
1. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:
   - In the drop-down, select **More**.
   - In the **Attach to an existing compute resource** pop-up, select the first drop-down. You will see a unique cluster name in that drop-down. Please select that cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:
1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.
1. Find the triangle icon to the right of your compute cluster name and click it.
1. Wait a few minutes for the cluster to start.
1. Once the cluster is running, complete the steps above to select your cluster.

## Requirements
Please review the following requirements before starting the lesson:
- To run this notebook, you need to use one of the following Databricks runtime(s): **17.3.x-cpu-ml-scala2.13**

## Classroom Setup
Before starting the Notebooks, run the provided classroom setup script. This script will define configuration variables necessary for the future demos and labs. Execute the following cell:


In [0]:
%run ../Includes/Classroom-Setup-0

**Other Conventions**

Throughout this lab, we'll refer to the object `DA`. This object, provided by Databricks Academy, contains variables such as your username, catalog name, schema name, working directory, and dataset locations. Run the code block below to view these details:

In [0]:
print(f"Username:          {DA.username}")
print(f"Catalog Name:      {DA.catalog_name}")
print(f"Schema Name:       {DA.schema_name}")
print(f"Working Directory: {DA.paths.working_dir}")
print(f"User DB Location:  {DA.paths.datasets}")


## Create a Landing Pad for the Credentials
A token is just like a username and password, so you should treat it with the same level of security as your own credentials. If you ever suspect a token has leaked, delete it immediately.

For the purpose of this training, we will create a landing pad in this notebook to record and store the credentials within the workspace. When using credentials in production, follow the security practices of your organization.

Run the following cell to create two text fields which you will populate in the next section:

- **Host:** The URL of the target workspace, which will form the base for all REST API endpoints. The framework will populate this value automatically using the current workspace, but this value can be overridden if desired.
- **Token:** A bearer token to authenticate with the target workspace.

## Generate Credentials

Create an authorization token for use with the Databricks CLI and API. If you are unable to create a token, please reach out to your workspace admin.

**Steps to Generate a Token:**

1. Click on your username in the top bar and select **User Settings** from the drop-down menu.
1. Click **User &gt; Developer**, then click **Access tokens &gt; Manage**.
1. Click **Generate new token**.
1. Specify the following:
   * A comment describing the purpose of the token (for example, *CLI Demo*).
   * The lifetime of the token; estimate the number of days you anticipate needing to complete this module.
1. Click **Generate**.
1. Copy the displayed token to the clipboard. You will not be able to view the token again; if you lose it, you will need to delete it and create a new one.
1. Click **Done**.
1. Paste the token into the **Token** field below.
1. If you are targeting a workspace other than the current one, paste it into the **Host** field. Otherwise, leave this value as-is.

In response to these inputs, these values will be recorded as follows:
* In the environment variables **`DATABRICKS_HOST`** and **`DATABRICKS_TOKEN`** so that they can be used for [authentication](https://docs.databricks.com/en/dev-tools/auth/index.html) by the Databricks CLI, APIs, and SDK that we use in subsequent notebooks
* Since environment variables are limited in scope to the current execution context, the values are persisted to a [file in your workspace](https://docs.databricks.com/en/files/workspace.html#) for use by subsequent notebooks

In [0]:
DA.get_credentials()

### Creating a Git Repository and Generating a Git Token

**1. Creating a Git Repository**

To create a Git repository for storing and managing your Databricks notebooks and configurations, follow these steps:

1. **Log in to GitHub:**
   - Go to [GitHub](https://github.com/) and log in with your credentials.

2. **Create a New Repository:**
   - Click on the **"+"** icon at the top-right corner and select **"New repository"**.

3. **Fill in Repository Details:**
   - **Repository name**: Provide a name for your repository, e.g., `Adv_mlops-ci-cd-demo`.
   - **Description** (optional): Add a description for your repository.
   - **Visibility**: Choose between **Public** or **Private**, depending on your preferences.
   - Check the box to **Add a README file** if you want to initialize the repository with a README.

4. **Create the Repository:**
   - Click the **"Create repository"** button.

**2. Generating a GitHub Personal Access Token**

To enable authentication between Databricks and GitHub, you’ll need to generate a Personal Access Token (PAT). Follow these steps:

1. **Go to GitHub Settings:**
   - Click on your profile picture at the top-right corner.
   - Select **"Settings"** from the dropdown menu.

2. **Navigate to Developer Settings:**
   - In the left-hand menu, scroll down and click on **"Developer settings"**.

3. **Select Personal Access Tokens:**
   - Click on **"Personal access tokens"**, then select **"Tokens (classic)"**.

4. **Generate a New Token:**
   - Click the **"Generate new token"** button and select **"Generate new Token (classic)"**.

5. **Specify Token Permissions:**
   - **Note**: If you are using fine-grained tokens (available in some accounts), ensure to match scopes accordingly.
   - Provide a **Note** for: `What's this token for?`, e.g., `Databricks Access`.
   - Set an **expiration date** for the token (e.g., 30 days, 90 days, etc.).
   - Select the following scopes:
     - **repo**: Full control of private repositories.
     - **workflow**: Read and write access to workflows.
     - **admin:repo_hook**: Manage repository hooks.
     - **delete_repo**: Delete repositories

6. **Generate the Token:**
   - Click **"Generate token"**.

7. **Copy the Token:**
   - **Important**: Copy the generated token immediately, as it won’t be shown again.

8. **Store the Token Securely:**
   - Save the token in a secure location or use it directly in your Databricks credentials setup.

**3. Using the GitHub Token in Databricks**

After generating the GitHub token, you can securely load it into Databricks using the widget interface or a configuration file. Refer to the relevant section in this notebook for detailed steps on loading Git credentials using the command DA.get_git_credentials().

To enable CI/CD functionalities, such as interacting with GitHub repositories, you need to provide the following GitHub credentials:

- **GitHub Username:** Your GitHub account username.

- **Repository Name:** The name of the repository you want to interact with.

- **GitHub Token:** The personal access token (PAT) you generated from GitHub.

To set or update these credentials, execute the following command:

> **Note**: Avoid hardcoding your token in the scripts for security purposes. Always use environment variables or secure credential storage.

In [0]:
DA.get_git_credentials()

&copy; 2026 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>